In [28]:
import os, csv, math
import pandas as pd
import numpy as np

grid_length = 8
grid_width = 4
channels_per_grid = 32


In [29]:

def calculate_average(lst):
    if not lst:
        return None

    total = sum(lst)
    average = total / len(lst)
    return average


def create_or_empty_file(file_path):
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            file.truncate(0)
        print(f"File '{file_path}' existed and has been emptied.")
    else:
        with open(file_path, 'w') as file:
            pass
        print(f"File '{file_path}' has been created.")
        

def get_filenames_with_keywords(folder_path, keywords):
    filenames = []
    for filename in os.listdir(folder_path):
        if any(keyword in filename for keyword in keywords):
            filenames.append(filename)
    return sorted(filenames)


def find_max_value_row_index(csv_file_path):
    
    max_value = float('-inf')  # Initialize with negative infinity
    max_row_index = -1

    with open(csv_file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row_index, row in enumerate(reader):
            try:
                column_10_value = float(row[13])  # Index 13 corresponds to the 14th column (0-based index)
                if column_10_value > max_value:
                    max_value = column_10_value
                    max_row_index = row_index
            except (ValueError, IndexError):
                # Skip rows with non-numeric or insufficient columns
                continue

    return max_row_index


def read_csv_as_list(csv_file):
    data = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            data.append(row)
    return data


def calculate_entropy(data):
    data_rms = np.array(data)
    rms_sq = data_rms ** 2
    sum_rms_sq = np.sum(rms_sq)
    probability = rms_sq / sum_rms_sq
    entropy = -np.sum(probability * np.log2(probability))
    return entropy


def calculate_coefficient_of_variation(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    coefficient_of_variation = (std / mean) * 100
    return coefficient_of_variation


def calculate_intensity(signal):
    
    if(np.sum(signal) > 0):
        intensity = math.log(np.sum(signal)/channels_per_grid, 10)
        return intensity
    else:
        return 0


def calculate_differential_intensity(csv_file, row_index, column_start, column_end):
    
    emg_data = np.genfromtxt(csv_file, delimiter=',')
    emg_row_data = emg_data[row_index, column_start:column_end]
    emg_diff_data = []
    
    for i in range(1,len(emg_row_data)):
        emg_diff_data.append(math.log(abs(emg_row_data[i] - emg_row_data[i-1]),10))
        
    return calculate_average(emg_diff_data)
        

def calculate_mean(signal):
    mean = np.mean(signal)
    return mean


def calculate_max(signal):
    max_value = np.max(signal)
    return max_value


def calculate_displacement(xcg1, ycg1, xcg2, ycg2):
    displacement = ((xcg2 - xcg1) ** 2 + (ycg2 - ycg1) ** 2) ** 0.5
    return displacement
    
    
def compute_EMG_parameters(csv_file, operation, target_row_index, column_start, column_end):

    data = read_csv_as_list(csv_file)

    element_list = []

    for col in range(column_start, column_end):
        
        element = float(data[target_row_index][col])
        element_list.append(element)
        
        #print(element_list)
        #param_value.append(operation(element_list))

    return operation(element_list)


def calculate_center_of_gravity(csv_file, row_index, column_start, column_end):
    
    emg_data = np.genfromtxt(csv_file, delimiter=',')
    emg_row_data = emg_data[row_index, column_start:column_end] 

    # Reshape the 1D array into a 2D grid (8x4)
    emg_grid = emg_row_data.reshape(grid_length, grid_width)

    # Calculate the CoG X and Y coordinates
    row_indices, col_indices = np.indices((grid_length, grid_width))
    total_rms = np.sum(emg_grid)
    cog_x = np.sum(col_indices * emg_grid) / total_rms
    cog_y = np.sum(row_indices * emg_grid) / total_rms

    return cog_x, cog_y



In [30]:
#labels = ['Participant', 'Arm', 'Exercise type', 'Entropy_1', 'CoV_1', 'Intensity_1', 'Differential Intensity_1', 'Mean_RMS_1', 'Max_RMS_1','Xcg_1', 'Ycg_1','Entropy_2', 'CoV_2', 'Intensity_2', 'Differential Intensity_2', 'Mean_RMS_2', 'Max_RMS_2', 'Xcg_2', 'Ycg_2']
labels = ['Participant', 'Arm', 'Exercise type', 'Entropy_1', 'CoV_1', 'Intensity_1', 'Differential Intensity_1', 'Mean_RMS_1', 'Max_RMS_1', 'Mean_ARV_1', 'Max_ARV_1','Xcg_1', 'Ycg_1','Entropy_2', 'CoV_2', 'Intensity_2', 'Differential Intensity_2', 'Mean_RMS_2', 'Max_RMS_2','Mean_ARV_2', 'Max_ARV_2', 'Xcg_2', 'Ycg_2']

Data_list = []
Indexes_list = []

folder_path = './EMG_RMS_Data'
folder_path_mdf = './EMG_MDF_Data'
folder_path_arv = './EMG_ARV_Data'

keywords = ["Push","Pull"]
matching_filenames = get_filenames_with_keywords(folder_path, keywords)


for filename in matching_filenames:
    
    print(filename)
    participant_list = filename.split("_")    
    csv_file = folder_path + "/" + filename
    csv_file_mdf = folder_path_mdf + "/" + filename
    csv_file_arv = folder_path_arv + "/" + filename
    
    peak_index = find_max_value_row_index(csv_file)
    print(peak_index)
    Indexes_list.append([filename, peak_index])
    
    participant_list.append(compute_EMG_parameters(csv_file, calculate_entropy, peak_index, 1, 33))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_coefficient_of_variation, peak_index, 1, 33))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_intensity, peak_index, 1, 33))
    participant_list.append(calculate_differential_intensity(csv_file, peak_index, 1, 33))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_mean, peak_index, 1, 33))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_max, peak_index, 1, 33))
    #participant_list.append(compute_EMG_parameters(csv_file_mdf, calculate_mean, peak_index, 1, 33))
    #participant_list.append(compute_EMG_parameters(csv_file_mdf, calculate_max, peak_index, 1, 33))
    participant_list.append(compute_EMG_parameters(csv_file_arv, calculate_mean, peak_index, 1, 33))
    participant_list.append(compute_EMG_parameters(csv_file_arv, calculate_max, peak_index, 1, 33))
    cogx_g1, cogy_g1 = calculate_center_of_gravity(csv_file, peak_index, 1, 33)
    participant_list.append(cogx_g1)
    participant_list.append(cogy_g1)
    
    participant_list.append(compute_EMG_parameters(csv_file, calculate_entropy, peak_index, 33, 65))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_coefficient_of_variation, peak_index, 33, 65))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_intensity, peak_index, 33, 65))
    participant_list.append(calculate_differential_intensity(csv_file, peak_index, 33, 65))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_mean, peak_index, 33, 65))
    participant_list.append(compute_EMG_parameters(csv_file, calculate_max, peak_index, 33, 65))
    #participant_list.append(compute_EMG_parameters(csv_file_mdf, calculate_mean, peak_index, 33, 65))
    #participant_list.append(compute_EMG_parameters(csv_file_mdf, calculate_max, peak_index, 33, 65))
    participant_list.append(compute_EMG_parameters(csv_file_arv, calculate_mean, peak_index, 33, 65))
    participant_list.append(compute_EMG_parameters(csv_file_arv, calculate_max, peak_index, 33, 65))
    cogx_g2, cogy_g2 = calculate_center_of_gravity(csv_file, peak_index, 33, 65)
    participant_list.append(cogx_g2)
    participant_list.append(cogy_g2)
    
    Data_list.append(participant_list)
    #print(len(participant_list))


df = pd.DataFrame(Data_list, columns=labels)
df_indexes = pd.DataFrame(Indexes_list, columns=['Filename','Indexes'])

csv_filename = 'Data_Functional.csv'
index_csv_filename = 'Functional_Indexes.csv'

create_or_empty_file(csv_filename)
create_or_empty_file(index_csv_filename)

df.to_csv(csv_filename, index=False) 
df_indexes.to_csv(index_csv_filename, index=False)


P01_Left_Pull1.csv
431
P01_Left_Push3.csv
415
P01_Right_Pull2.csv
394
P01_Right_Push1.csv
287
P02_Left_Pull3.csv
537
P02_Left_Push2.csv
459
P02_Right_Pull3.csv
332
P02_Right_Push1.csv
446
P03_Left_Pull.csv
533
P03_Left_Push3.csv
364
P03_Right_Pull.csv
544
P03_Right_Push2.csv
417
P04_Left_Pull1.csv
293
P04_Left_Push1.csv
357
P04_Right_Pull2.csv
363
P04_Right_Push2.csv
503
P05_Left_Pull3.csv
367
P05_Left_Push3.csv
473
P05_Right_Pull2.csv
263
P05_Right_Push3.csv
440
P06_Left_Pull2.csv
358
P06_Left_Push3.csv
441
P06_Right_Pull1.csv
352
P06_Right_Push3.csv
428
P08_Left_Pull3.csv
309
P08_Left_Push3.csv
457
P08_Right_Pull1.csv
235
P08_Right_Push2.csv
331
P09_Left_Pull1.csv
522
P09_Left_Push2.csv
362
P09_Right_Pull1.csv
404
P09_Right_Push2.csv
444
P10_Left_Pull1.csv
334
P10_Left_Push2.csv
379
P10_Right_Pull3.csv
363
P10_Right_Push2.csv
452
P11_Left_Pull2.csv
386
P11_Left_Push3.csv
378
P11_Right_Pull3.csv
333
P11_Right_Push3.csv
470
P12_Left_Pull1.csv
260
P12_Left_Push3.csv
319
P12_Right_Pull3.